In [1]:
import os
os.environ["OPENAI_API_KEY"] = 'sk-proj-NnacoAq1exHhxjM8PgSkT3BlbkFJHOYMFIiDA4FIUWkLk2PX'

In [4]:
import json
filename = 'Extracted_Data/Medical_Record_File_3.json'
data = None
with open(filename, 'r') as file:
    data = json.load(file)

In [5]:
data

{'1': 'ExamWorks\nClaimant: Juan Lopez, Jr\nTable of Contents\nSorted Medical Records 2\nJohnson, DC, LAc, John 2\n2/9/2024 - IME 2\nUnis, MD, Douglas 9\n2/2/2024 - IME 9\nVLI Medical, PC 15\n12/14/2023 15\n12/14/2023 - Outcome Assessment 20\n1/30/2024 26\n1/30/2024 - Outcome Assessment 31\n2/24/2024 37\nPhysical Therapy Notes/Meds 42\nUnknown Facility 42\n12/5/2023-3/11/2024 42\nChiropractic Records 43\nVLI Medical, PC 43\n10/12/2023-2/15/2024 43 \n',
 '2': 'John E. Johnson Jr., D.C., L.Ac.\nLicensed Chiropractor & Licensed in Acupuncture\n717 Church Avenue\nBrooklyn, NY 11218\nTel: (718) 421-4800\nFax: (718) 421-4815\nFebruary 9, 2024\nRe: Juan Lopez\nClaim #: SLU75209\nDate of Accident: 2-28-23\nCase #: 22215951\nTo Whom It May Concern:\nAs per your request, I performed a chiropractic and acupuncture re-examination on the above-named claimant. This examination was performed at my Brooklyn office on February 9, 2024. A member of our staff was present during the examination. Photo ide

In [6]:
result = ''
def flatten_json(obj, path=''):
    global result
    if isinstance(obj, dict):
        for key, value in obj.items():
            new_path = f"{path}.{key}" if path else key
            flatten_json(value, new_path)
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            new_path = f"{path}[{index}]"
            flatten_json(item, new_path)
    else:
        result += f"{path}: {obj}\n"

flatten_json(data)

In [8]:
print(result)

1: ExamWorks
Claimant: Juan Lopez, Jr
Table of Contents
Sorted Medical Records 2
Johnson, DC, LAc, John 2
2/9/2024 - IME 2
Unis, MD, Douglas 9
2/2/2024 - IME 9
VLI Medical, PC 15
12/14/2023 15
12/14/2023 - Outcome Assessment 20
1/30/2024 26
1/30/2024 - Outcome Assessment 31
2/24/2024 37
Physical Therapy Notes/Meds 42
Unknown Facility 42
12/5/2023-3/11/2024 42
Chiropractic Records 43
VLI Medical, PC 43
10/12/2023-2/15/2024 43 

2: John E. Johnson Jr., D.C., L.Ac.
Licensed Chiropractor & Licensed in Acupuncture
717 Church Avenue
Brooklyn, NY 11218
Tel: (718) 421-4800
Fax: (718) 421-4815
February 9, 2024
Re: Juan Lopez
Claim #: SLU75209
Date of Accident: 2-28-23
Case #: 22215951
To Whom It May Concern:
As per your request, I performed a chiropractic and acupuncture re-examination on the above-named claimant. This examination was performed at my Brooklyn office on February 9, 2024. A member of our staff was present during the examination. Photo identification was presented prior to this ex

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def generate_tokens(s):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
  # splits = text_splitter.split_text(s)
  return text_splitter.split_text(s) #this should return the list of documents.

# Doc texts split
from langchain_text_splitters import RecursiveCharacterTextSplitter

# chunk_size_tok = 2000
# text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=chunk_size_tok, chunk_overlap=200
# )
# texts_split = text_splitter.split_documents(result) 
# texts_split

texts_split = generate_tokens(result)

In [40]:
texts_split[0:5]

['1: ExamWorks\nClaimant: Juan Lopez, Jr\nTable of Contents\nSorted Medical Records 2\nJohnson, DC, LAc, John 2\n2/9/2024 - IME 2\nUnis, MD, Douglas 9\n2/2/2024 - IME 9\nVLI Medical, PC 15\n12/14/2023 15\n12/14/2023 - Outcome Assessment 20\n1/30/2024 26\n1/30/2024 - Outcome Assessment 31\n2/24/2024 37\nPhysical Therapy Notes/Meds 42\nUnknown Facility 42\n12/5/2023-3/11/2024 42\nChiropractic Records 43\nVLI Medical, PC 43\n10/12/2023-2/15/2024 43',
 '2: John E. Johnson Jr., D.C., L.Ac.\nLicensed Chiropractor & Licensed in Acupuncture\n717 Church Avenue\nBrooklyn, NY 11218\nTel: (718) 421-4800\nFax: (718) 421-4815\nFebruary 9, 2024\nRe: Juan Lopez\nClaim #: SLU75209\nDate of Accident: 2-28-23\nCase #: 22215951\nTo Whom It May Concern:\nAs per your request, I performed a chiropractic and acupuncture re-examination on the above-named claimant. This examination was performed at my Brooklyn office on February 9, 2024. A member of our staff was present during the examination. Photo identifica

In [32]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain3 = load_summarize_chain(llm, chain_type="refine")

chain3.run(texts_split)

"The patient, Juan Lopez, received trigger point/nerve block injections targeting specific muscles in the head, neck, lumbar paraspinal, upper back, shoulder, and other areas. The injections were well-tolerated, with the patient experiencing only mild lightheadedness. The treatment plan includes bed rest, avoiding physical activity, and attending supervised physical therapy on a regular basis, with the frequency to be determined. The patient reported decreased muscle stiffness and pain following the procedure. The prognosis for recovery is cautiously optimistic, and the disability resulting from the injuries sustained in the accident may be of a temporary or permanent nature. The patient's medical history includes subjective and objective findings related to knee pain, neck pain, tenderness, and vertigo, among other symptoms."

In [34]:
from langchain import PromptTemplate
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": docs}, return_only_outputs=True)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-1106 in organization org-AzxkHbslquofpvZHMHVgJn0V on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}